In [1]:
def intersects(start, length, starts, ends):
    end = start + length
    for i in range(len(starts)):
        ranges = range(starts[i], ends[i] + 1)
        if (start in ranges or end in ranges):
            return True
    return False

In [2]:
def isDisorder(start, length, starts, ends):
    end = start + length
    for i in range(len(starts)):
        if start >= starts[i] and end <= ends[i]:
            return True
    return False

In [3]:
from typing import List

def update_rolling_hash(current_hash: int, old_char: str, new_char: str, window_size: int, base: int = 31, mod: int = 10**9 + 9) -> int:

    current_hash = (current_hash - ord(old_char) * pow(base, window_size - 1, mod)) % mod
    current_hash = (current_hash * base + ord(new_char)) % mod
    
    return current_hash if current_hash >= 0 else current_hash + mod

In [4]:
def calculate_initial_hash(s: str, window_size: int, base: int = 31, mod: int = 10**9 + 9) -> int:
    current_hash = 0
    for i in range(window_size):
        current_hash = (current_hash * base + ord(s[i])) % mod
    return current_hash

In [5]:
def intersects(start, length, starts, ends):
    end = start + length
    for i in range(len(starts)):
        ranges = range(starts[i], ends[i] + 1)
        if (start in ranges or end in ranges):
            return True
    return False

In [5]:
from Bio import SeqIO
import numpy as np
import json

def extractSequences(filename, outfile, data):
    fasta_sequences = SeqIO.parse(open(filename), 'fasta')

    with open(data) as f:
        podaci = json.load(f)

    j = 0
    sequenceMap = {}
    with open(outfile, "w") as out:
        for fasta in fasta_sequences:
            name, sequence = fasta.id, str(fasta.seq)
            regions = podaci[j]['disprot_consensus']['Structural state']
            j += 1 
            if name == "DP00072":
                continue

            starts, ends = [], []
            for region in regions:
                if region["type"] == "D":
                    starts.append(region["start"])
                    ends.append(region["end"])

            seq_len = len(sequence)
            print(name)
            for n in range(3, seq_len + 1):
                # Compute the initial hash for the first window of size n
                if seq_len >= n:
                    current_hash = calculate_initial_hash(sequence[:n], n)
                    if current_hash not in sequenceMap:
                        sequenceMap[current_hash] = np.array([0, 0, 0])

                    if isDisorder(0, n, starts, ends):
                        sequenceMap[current_hash][0] += 1
                    elif intersects(0, n, starts, ends):
                        sequenceMap[current_hash][1] += 1
                    else:
                        sequenceMap[current_hash][2] += 1

                    # Sliding window with rolling hash
                    for i in range(1, seq_len - n + 1):
                        current_hash = update_rolling_hash(current_hash, sequence[i - 1], sequence[i + n - 1], n)
                        if current_hash not in sequenceMap:
                            sequenceMap[current_hash] = np.array([0, 0, 0])
                        
                        if isDisorder(i, n, starts, ends):
                            sequenceMap[current_hash][0] += 1
                        elif intersects(i, n, starts, ends):
                            sequenceMap[current_hash][1] += 1
                        else:
                            sequenceMap[current_hash][2] += 1

                    if len(sequenceMap) >= 20000:
                        for key, value in sequenceMap.items():  
                            out.write(f'{key}:{value}\n')
                        sequenceMap.clear()

            starts.clear()
            ends.clear()
        
        for key, value in sequenceMap.items():  
            out.write(f'{key}:{value}\n')

In [6]:
def writeSequencesToFile(inFiles, outFiles, data):
    for i in range(len(inFiles)):
        extractSequences(inFiles[i], outFiles[i], data[i])

In [10]:
outFiles = ["../sequences/sequences1proba.txt", "../sequences/sequences2proba.txt", "../sequences/sequences3proba.txt", "../sequences/sequences4proba.txt", "../sequences/sequences5proba.txt"]
inFiles = ["../preprocessedFiles/preprocessed1.fasta", "../preprocessedFiles/preprocessed2.fasta", "../preprocessedFiles/preprocessed3.fasta", "../preprocessedFiles/preprocessed4.fasta", "../preprocessedFiles/preprocessed5.fasta"]
data = ["../trainSets/trainSet1.json", "../trainSets/trainSet2.json", "../trainSets/trainSet3.json", "../trainSets/trainSet4.json", "../trainSets/trainSet5.json"]
writeSequencesToFile(inFiles, outFiles, data)

DP03718
999969226
999974992
999969412
999224451
999051471
970595300


In [7]:
def extractSequences(filename, outfile, data):
    fasta_sequences = SeqIO.parse(open(filename), 'fasta')

    with open(data) as f:
        podaci = json.load(f)

    j = 0
    sequenceMap = {}
    with open(outfile, "w") as out:
        for fasta in fasta_sequences:
            name, sequence = fasta.id, str(fasta.seq)
            regions = podaci[j]['disprot_consensus']['Structural state']
            j += 1 
            if name != "DP00072":
                continue

            starts, ends = [], []
            for region in regions:
                if region["type"] == "D":
                    starts.append(region["start"])
                    ends.append(region["end"])

            seq_len = len(sequence)
            print(name)
            for n in range(3, seq_len + 1):
                # Compute the initial hash for the first window of size n
                if seq_len >= n:
                    current_hash = calculate_initial_hash(sequence[:n], n)
                    if current_hash not in sequenceMap:
                        sequenceMap[current_hash] = np.array([0, 0, 0])

                    if isDisorder(0, n, starts, ends):
                        sequenceMap[current_hash][0] += 1
                    elif intersects(0, n, starts, ends):
                        sequenceMap[current_hash][1] += 1
                    else:
                        sequenceMap[current_hash][2] += 1

                    # Sliding window with rolling hash
                    for i in range(1, seq_len - n + 1):
                        current_hash = update_rolling_hash(current_hash, sequence[i - 1], sequence[i + n - 1], n)
                        if current_hash not in sequenceMap:
                            sequenceMap[current_hash] = np.array([0, 0, 0])
                        
                        if isDisorder(i, n, starts, ends):
                            sequenceMap[current_hash][0] += 1
                        elif intersects(i, n, starts, ends):
                            sequenceMap[current_hash][1] += 1
                        else:
                            sequenceMap[current_hash][2] += 1

                    if len(sequenceMap) >= 20000:
                        for key, value in sequenceMap.items():  
                            out.write(f'{key}:{value}\n')
                        sequenceMap.clear()

            starts.clear()
            ends.clear()
        
        for key, value in sequenceMap.items():  
            out.write(f'{key}:{value}\n')

In [ ]:
outFiles = ["../sequences/72proba.txt"]
inFiles = ["../preprocessedFiles/preprocessed1.fasta"]
data = ["../trainSets/trainSet1.json"]
writeSequencesToFile(inFiles, outFiles, dat